# 9章　特徴量抽出による次元削減 
## レシピ9.1　主成分を用いた特徴量削減 


In [ ]:
# ライブラリをロード
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets

# データをロード
digits = datasets.load_digits()

# 特徴量行列を標準化
features = StandardScaler().fit_transform(digits.data)

# 分散を99%維持するPCAを作成
pca = PCA(n_components=0.99, whiten=True)

# PCAを実行
features_pca = pca.fit_transform(features)

# 結果を表示
print("もとの特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_pca.shape[1])


## レシピ9.2　データが線形分離不可能な際の特徴量削減 


In [ ]:
# ライブラリをロード
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles

# 線形分離不可能なデータを生成
features, _ = make_circles(n_samples=1000, random_state=1, noise=0.1, factor=0.1)

# RBF（radius basis function）カーネルPCAを適用
kpca = KernelPCA(kernel="rbf", gamma=15, n_components=1)
features_kpca = kpca.fit_transform(features)
print("もとの特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_kpca.shape[1])

## レシピ9.3　クラスの分離性最大化による特徴量削減 


In [ ]:
# ライブラリをロード
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Irisデータセットをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# LDAを作成して実行し、さらにそれを用いて特徴量を変換
lda = LinearDiscriminantAnalysis(n_components=1)
features_lda = lda.fit(features, target).transform(features)

# 特徴量数を表示
print("もとの特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_lda.shape[1])

In [ ]:
lda.explained_variance_ratio_

In [ ]:
lda.explained_variance_ratio_

In [ ]:
# LDAを作成して実行
lda = LinearDiscriminantAnalysis(n_components=None)
features_lda = lda.fit(features, target)

# 寄与率の配列を作成
lda_var_ratios = lda.explained_variance_ratio_

# 関数を定義
def select_n_components(var_ratio, goal_var):
    
    # 説明された寄与率の累計変数を初期化
    total_variance = 0.0

    # 特徴量数の初期値を設定
    n_components = 0

    # それぞれの特徴量の寄与率に対して
    for explained_variance in var_ratio:
        # 寄与率を累計に加算
        total_variance += explained_variance

        # 成分数に1足す
        n_components += 1

        # 説明された寄与率累計が目標値に到達していたら
        if total_variance >= goal_var:
            # ループ終了
            break

    # 成分（特徴量）数を返す
    return n_components

# 関数を実行
select_n_components(lda_var_ratios, 0.95)

## レシピ9.4　行列因子分解による特徴量削減 


In [ ]:
# ライブラリをロード
from sklearn.decomposition import NMF
from sklearn import datasets

# データをロード
digits = datasets.load_digits()

# データを特徴量行列として使用
features = digits.data

# NMFを作成、訓練して適用
nmf = NMF(n_components=10, random_state=1)
features_nmf = nmf.fit_transform(features)

# 結果を表示
print("もとの特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_nmf.shape[1])

## レシピ9.5　疎データの特徴量削減 


In [ ]:
# ライブラリをロード
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn import datasets
import numpy as np

# データをロード
digits = datasets.load_digits()

# 特徴量行列の標準化
features = StandardScaler().fit_transform(digits.data)

# 疎行列の作成
features_sparse = csr_matrix(features)

# TSVDの作成
tsvd = TruncatedSVD(n_components=10)

# 疎行列に対してTSVDを実行
features_sparse_tsvd = tsvd.fit(features_sparse).transform(features_sparse)

# 結果を表示
print("もとの特徴量数:", features_sparse.shape[1])
print("削減後の特徴量数:", features_sparse_tsvd.shape[1])


In [ ]:
# 最初の3つの成分の寄与率の総計
tsvd.explained_variance_ratio_[0:3].sum()

In [ ]:
# 特徴量数-1を指定してTSVDを作成して実行
tsvd = TruncatedSVD(n_components=features_sparse.shape[1]-1)
features_tsvd = tsvd.fit(features)

# 説明された分散の割合のリスト
tsvd_var_ratios = tsvd.explained_variance_ratio_

# 関数を定義
def select_n_components(var_ratio, goal_var):
    # 説明された寄与率の累計変数を初期化
    total_variance = 0.0

    # 特徴量数の初期値を設定
    n_components = 0

    # それぞれの特徴量の寄与率に対して
    for explained_variance in var_ratio:

        # 寄与率を累計に加算
        total_variance += explained_variance

        # 成分数に1足す
        n_components += 1

        # 説明された寄与率累計が目標値に到達していたら
        if total_variance >= goal_var:
            # ループを終了
            break
        
    # 成分（特徴量）数を返す
    return n_components

# 関数を実行
select_n_components(tsvd_var_ratios, 0.95)